In [35]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException    
from datetime import date, timedelta
import time
import re
from user_agent import generate_user_agent

In [38]:
headers = {'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))}
opts = webdriver.ChromeOptions()
opts.add_argument({'User-Agent': generate_user_agent(device_type="desktop", os=('mac', 'linux'))})

In [41]:
driver = webdriver.Chrome()

In [42]:
driver.get('https://www.google.com')

# Find search box, search for term
search_box = driver.find_element_by_name('q')
search_box.send_keys('Flüchtlinge\n')


tools = driver.find_element_by_xpath('//*[@id="hdtb-tls"]')
tools.click()
time.sleep(4)

# setting up the timeframe I want my results to be in

start = date(2014, 1, 1)
week = timedelta(days=7)
next_day = timedelta(days=1)
end = date(2014, 1, 31)
scrape_start_date = start
ending = scrape_start_date+week

# Empty list to store the links in. 
links = []

# This while loop tells Selenium to scrape until the end of the year 2014
# on a week-by-week basis. 

while scrape_start_date < end:
    print('---new week ---')
    start_time = str(scrape_start_date.strftime('%m/%d/%Y'))
    end_time = str(ending.strftime('%m/%d/%Y'))
    print(start_time)
    print(end_time)
    
    find_time = driver.find_elements_by_class_name('mn-hd-txt')[1]
    find_time.click()
    set_time = driver.find_element_by_css_selector('span.q')
    set_time.click()
    time_begin = driver.find_element_by_css_selector('input#cdr_min')
    time_end = driver.find_element_by_css_selector('input#cdr_max')

    time_begin.clear()
    time_end.clear()
    time.sleep(0.5)
#    time_begin.send_keys(Keys.CONTROL + "a")
#    time_begin.send_keys(Keys.BACKSPACE)
    time_begin.send_keys(start_time)

#    time_begin.send_keys(Keys.BACKSPACE)
    time_end.send_keys(end_time)
    time.sleep(1)
    # Selenium now knows what week I'm looking for. 
    # Time to click the button, start the search.
    begin_search = driver.find_elements_by_class_name("cdr_go")[1]
    begin_search.click()

    urls = driver.find_elements_by_css_selector('h3.r a')
    for url in urls:
        links.append(url.get_attribute('href'))
    next_page = driver.find_element_by_css_selector('#pnnext')
    
    while next_page != None:
        try:
            next_page = driver.find_element_by_css_selector('#pnnext')
            next_page.click()
            time.sleep(5)
            urls = driver.find_elements_by_css_selector('h3.r a')
            for url in urls:
                links.append(url.get_attribute('href'))
        except NoSuchElementException:
            print('Done with this. Date is', end_time)
            break    
    scrape_start_date = ending + next_day
    ending = scrape_start_date + week

---new week ---
01/01/2014
01/08/2014
Done with this. Date is 01/08/2014
---new week ---
01/09/2014
01/16/2014
Done with this. Date is 01/16/2014
---new week ---
01/17/2014
01/24/2014
Done with this. Date is 01/24/2014
---new week ---
01/25/2014
02/01/2014
Done with this. Date is 02/01/2014


In [43]:
df = pd.DataFrame(links)

In [44]:
df.rename(columns={0: "links"}, inplace=True)
df.

,links
0,https://www.zeit.de/gesellschaft/zeitgeschehen...
1,https://www.op-online.de/region/neu-isenburg/f...
2,http://www.spiegel.de/politik/ausland/israel-f...
3,https://www.uno-fluechtlingshilfe.de/news/syri...
4,http://www.taz.de/!5051301/
5,https://www.abendblatt.de/hamburg/article12356...
6,https://www.berliner-zeitung.de/geschaeftsmann...
7,https://www.landkreis-esslingen.de/site/LRA-Es...
8,http://www.bamf.de/SharedDocs/Anlagen/DE/Downl...
9,https://www.youtube.com/watch?v=TOq0mNMeN0E
